In [2]:
import pandas as pd

In [3]:
# metadata
uw_hifi = pd.read_csv('sra_biosample_UW_HiFi_Y4_AND_Y3_Topoff_metadata.tsv',sep='\t')

# aws
uw_aws = pd.read_csv('UW_HPRC_HiFi_Y4_AND_Y3_Topoff.transfer_ec2.csv',header=None)
uw_aws['filename'] = [sample.split('/')[-1] for sample in uw_aws[1].tolist()]
uw_aws.rename(columns={1: 'path'}, inplace=True)

# readstats
uw_readstats = pd.read_csv('UW_HPRC_HiFi_Y4_AND_Y3_Topoff_sample_file_hifi_qc_aggregate.csv')
uw_readstats['filename'] = ['_'.join(sample.split('_')[1:]) for sample in uw_readstats.filename.tolist()]
# uw_readstats['filename'] = uw_readstats['filename'].str.replace(r'\.bam$', '', regex=True)

uw_y4 = pd.merge(uw_hifi, uw_aws, on='filename', how='inner')
uw_y4.drop(columns=[0], inplace=True)

uw_assembly = pd.merge(uw_y4, uw_readstats, on='filename',how='inner')

uw_sra_meta = pd.read_csv('metadata-14548217-processed-ok.tsv',sep='\t')
uw_assembly = pd.merge(uw_assembly,uw_sra_meta[['filename','accession']], on='filename', how='inner')


# note the PG identifier needs to be manually resolved
assert uw_hifi.shape[0] == uw_readstats.shape[0] == uw_y4.shape[0] == uw_assembly.shape[0] == uw_sra_meta.shape[0]

In [5]:
uw_aws['sample_ID'] = [sample.split('/')[5] for sample in uw_aws['path'].tolist()]

In [6]:
uw_assembly['sample_ID'] = [sample.split('.')[0] for sample in uw_assembly['library_ID'].tolist()]
uw_assembly['sample_ID'] = [file_name.replace('GM', 'NA') for file_name in uw_assembly['sample_ID'].tolist()]
uw_assembly['sample_ID'] = [file_name.replace('PG', 'HG') for file_name in uw_assembly['sample_ID'].tolist()]

,0,path,filename,sample_ID
0,s3://human-pangenomics/submissions/4f8c8e63-df...,s3://human-pangenomics/working/HPRC/HG00097/ra...,m84046_230716_051716_s3.fail_reads.bc2086.bam,HG00097
1,s3://human-pangenomics/submissions/4f8c8e63-df...,s3://human-pangenomics/working/HPRC/HG00097/ra...,m84046_230716_051716_s3.hifi_reads.bc2086.bam,HG00097
2,s3://human-pangenomics/submissions/4f8c8e63-df...,s3://human-pangenomics/working/HPRC/HG00097/ra...,m84046_230716_054822_s4.fail_reads.bc2086.bam,HG00097
3,s3://human-pangenomics/submissions/4f8c8e63-df...,s3://human-pangenomics/working/HPRC/HG00097/ra...,m84046_230716_054822_s4.hifi_reads.bc2086.bam,HG00097
4,s3://human-pangenomics/submissions/4f8c8e63-df...,s3://human-pangenomics/working/HPRC/HG00097/ra...,m84046_230718_223822_s1.fail_reads.bc2086.bam,HG00097


In [10]:
uw_assembly.to_csv('UW_HPRC_HiFi_Y4_AND_Y3_Topoff_final_table.csv',index=False)